In [33]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
from utils import process_runner_books
import pytz
import constants


# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = constants.USERNAME
my_password = constants.PASSWORD
# my_app_key = constants.API_KEY_DEMO #demo
my_app_key = constants.API_KEY_LIVE #live

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [ ]:
## TODO
liability_amount = 1500

In [13]:
#######################################
# Filter out just greyhoud racing
#######################################
gh_racing_id = constants.GH_RACING_ID #Greyhound racing ID
greyhound_racing_filter = filters.market_filter(
    event_type_ids=[gh_racing_id],
    market_countries=['GB'],
    market_type_codes=['FORECAST'],
    market_start_time={
        'to': (datetime.datetime.utcnow() + datetime.timedelta(hours=6)).strftime("%Y-%m-%dT%TZ") #sydtime 5pm to 7am next day for GB gh races
    }
)

#  This returns a list
gb_gh_events = trading.betting.list_events(
    filter=greyhound_racing_filter)

# Create a DataFrame with all the events by iterating over each event object
gb_gh_events_df = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in gb_gh_events],
    'Event ID': [event_object.event.id for event_object in gb_gh_events],
    'Event Venue': [event_object.event.venue for event_object in gb_gh_events],
    'Country Code': [event_object.event.country_code for event_object in gb_gh_events],
    'Time Zone': [event_object.event.time_zone for event_object in gb_gh_events],
    'Open Date': [event_object.event.open_date for event_object in gb_gh_events],
    'Market Count': [event_object.market_count for event_object in gb_gh_events]
})

gb_gh_events_df

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Swindon (F/C) 20th Jun,31539184,Swindon,GB,Europe/London,2022-06-20 10:13:00,2
1,Yarmouth (F/C) 20th Jun,31539120,Yarmouth,GB,Europe/London,2022-06-20 17:19:00,6
2,Monmore (F/C) 20th Jun,31539074,Monmore,GB,Europe/London,2022-06-20 12:49:00,14
3,Doncaster (F/C) 20th Jun,31539138,Doncaster,GB,Europe/London,2022-06-20 17:09:00,6
4,Romford (F/C) 20th Jun,31539097,Romford,GB,Europe/London,2022-06-20 12:57:00,14
5,Perry Barr (F/C) 20th Jun,31539048,Perry Barr,GB,Europe/London,2022-06-20 10:06:00,2
6,Sheffield (F/C) 20th Jun,31539067,Sheffield,GB,Europe/London,2022-06-20 13:04:00,12
7,Towcester (F/C) 20th Jun,31539147,Towcester,GB,Europe/London,2022-06-20 17:17:00,6
8,Nottingham (F/C) 20th Jun,31539135,Nottingham,GB,Europe/London,2022-06-20 17:26:00,5


In [14]:
#######################################
#TODO: extract a list of IDs for the forecast markets
#######################################
fc_venue_ids = []
fc_venue_names = []
for eventObj in gb_gh_events:
    fc_venue_ids.append(eventObj.event.id)
    fc_venue_names.append(eventObj.event.name)

fc_venue_ids, fc_venue_names


(['31539184',
  '31539120',
  '31539074',
  '31539138',
  '31539097',
  '31539048',
  '31539067',
  '31539147',
  '31539135'],
 ['Swindon (F/C) 20th Jun',
  'Yarmouth (F/C) 20th Jun',
  'Monmore (F/C) 20th Jun',
  'Doncaster (F/C) 20th Jun',
  'Romford (F/C) 20th Jun',
  'Perry Barr (F/C) 20th Jun',
  'Sheffield (F/C) 20th Jun',
  'Towcester (F/C) 20th Jun',
  'Nottingham (F/C) 20th Jun'])

In [15]:
#######################################
## Filter out the next 5 upcoming games
#######################################
market_catalogue_filter = filters.market_filter(
    event_ids=fc_venue_ids)

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    market_projection= ['MARKET_START_TIME', 'EVENT'],
    max_results='5',
    sort='FIRST_TO_START'
)

market_types_venueOfTheDay_df = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Market Start Time': [market_cat_object.market_start_time for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues],
    'Venue': [market_cat_object.event.venue for market_cat_object in market_catalogues]
    
})

market_types_venueOfTheDay_df

,Market Name,Market ID,Market Start Time,Total Matched,Venue
0,Forecast,1.200327152,2022-06-20 12:36:00,2196.693736,Perry Barr
1,Forecast,1.200330701,2022-06-20 12:44:00,891.684072,Swindon
2,Forecast,1.200327554,2022-06-20 12:49:00,123.888880,Monmore
3,Forecast,1.200327153,2022-06-20 12:51:00,0.000000,Perry Barr
4,Forecast,1.200328143,2022-06-20 12:57:00,0.000000,Romford


In [16]:
#######################################
## Check the start time of the next game
## and sleep until 15 seconds before the game
#######################################
timeNow = (datetime.datetime.now(pytz.timezone("Europe/London"))-datetime.timedelta(hours=1)) ## minus 1 hr due to daylight savings maybe?
print(timeNow.strftime("%Y-%m-%d %T"))
## TODO Compare the list of markets - if timeNow is less than market start time (ie. market hasn't started yet), pick it. Otherwise, skip
time1 = timeNow.replace(tzinfo=pytz.UTC)

for marketObj in market_catalogues:
    time2 = marketObj.market_start_time.replace(tzinfo=pytz.UTC)
    if time1 < time2:
        ## Found the race to bet
        myRaceID = marketObj.market_id # store the market id
        myRaceVenue = marketObj.event.venue.lower().replace(" ", "-") # store the venue
        print("Found the market to lay: Name = " + myRaceVenue + " id = " + str(myRaceID))
        break

##TODO Sleep until 15 seconds before the start time


##TODO Choose the liability amount



2022-06-20 12:48:41
Found the market to lay: Name = monmore id = 1.200327554


In [24]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

#TODO: extract a list of IDs for the lay options
lay_options_ids = []
for obj in market_book.runners:
    lay_options_ids.append(obj.selection_id)

print(lay_options_ids)

[41419585, 41419586, 41419587, 41419588, 41419589, 41419592, 41419593, 41419594, 41419595, 41419596, 41419599, 41419600, 41419601, 41419602, 41419603, 41419606, 41419607, 41419608, 41419609, 41419610, 41419613, 41419614, 41419615, 41419616, 41419617, 41419620, 41419621, 41419622, 41419623, 41419624]


In [25]:
#######################################
# Get odds from NEDS
#######################################
from utils import choose_lay_option

lay_selection_index = choose_lay_option(myRaceVenue)

if(lay_selection_index == -1):
    print("ERROR!!! 2 same odds found")
    assert()
    
elif(lay_selection_index == -2):
    print("ERROR!!! less than 6 runners")
    assert()

elif(lay_selection_index == -3):
    print("ERROR!!! NEDS API failed")
    assert()
else:
    print("Lay option found successfully")
    print(lay_selection_index)

lay_selection_id = lay_options_ids[lay_selection_index]
print(lay_selection_id)
fav_price = runners_df.loc[runners_df['Best Lay Price'].idxmin(), 'Best Lay Price']

limit_order_filter = filters.limit_order(
    price=fav_price,
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=liability_amount # use this if i want $1500 as liability
)

instructions_filter = filters.place_instruction(
    selection_id = str(lay_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

True Venue Name = monmore
['4.80', '4.00', '4.00', '3.60', '4.60', '7.50']
['3.60', '4.00', '4.00', '4.60', '4.80', '7.50']
Forecast = 4-2
Two of the 3 smallest odds are the same : pos1 = 3,pos2 = 1,pos3 = 1
ERROR!!! 2 same odds found
41419624


{'orderType': 'LIMIT',
 'selectionId': '41419624',
 'side': 'LAY',
 'limitOrder': {'price': 11.0,
  'persistenceType': 'LAPSE',
  'betTargetType': 'PAYOUT',
  'betTargetSize': 1500}}

In [26]:
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [35]:
# listClearedOrders
# cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED",
#                                                     market_ids=[myRaceID])

# # 
# # Create a DataFrame from the orders
# pd.DataFrame(cleared_orders._data['clearedOrders'])

,eventTypeId,eventId,marketId,selectionId,handicap,betId,placedDate,persistenceType,orderType,side,betOutcome,priceRequested,settledDate,lastMatchedDate,betCount,priceMatched,priceReduced,sizeSettled,profit,customerStrategyRef
0,4339,31539184,1.200330702,41419621,0.0,271954723076,2022-06-20T12:57:29.000Z,LAPSE,LIMIT,LAY,WON,15.0,2022-06-20T13:04:48.000Z,2022-06-20T12:58:09.000Z,1,15.0,False,100.0,100.0,Naing_maker
1,4339,31539184,1.200330702,41419621,0.0,271954787771,2022-06-20T12:58:33.000Z,LAPSE,LIMIT,LAY,WON,16.0,2022-06-20T13:04:48.000Z,2022-06-20T12:58:33.000Z,1,16.0,False,6.0,6.0,NaN
